In [9]:
library(data.table)
library(ggplot2)
library(dplyr)
library(viridis)
library(qqman)
library(qvalue)
library(topGO)
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0



Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: viridisLite

For example usage please run: vignette('qqman')

Citation appreciated but not required:
Turner, (2018). qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. Journal of Open Source Software, 3(25), 731, https://doi.org/10.21105/joss.00731.

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    comb

In [ ]:
#check out angsd results
#hwe <- fread("~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/appc_angsd.hwe.gz")
#setnames(hwe, 9, "p")
#nrow(hwe)

In [ ]:
#plot(hwe$hweFreq, hwe$Freq) #no super obvious het problems

__make a sites file to filter out linked loci with pcangsd__

In [ ]:
#how do maf and calls across ind look- also make sites files for angsd and filter sites for pcangsd
sites <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/appc_angsd_probs.hwe.gz")
#head(sites)
unlinked <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_unlinked_for_angsd.txt")
setnames(unlinked, 1:2, c("Chromo", "Position"))
unlinked[, filter := 1] #when ngsld has kept the site, mark with 1 so pcangsd knows to keep it
test_filter_sites <- left_join(sites, unlinked) #join with full sites dt and mark linked  sites with 0
#head(test_filter_sites)
#nrow(test_filter_sites)==nrow(sites) #check that this is true
test_filter_sites[is.na(filter), filter := 0]
#write out for pcangsd
#fwrite(test_filter_sites[,.(filter)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/filter_linked_sites_pcangsd.txt", col.names = F)

__make positions file for angsd to restrict to unlinked sites__

In [ ]:
nrow(maf[, .N, by="chromo"])
pyam <- unique(maf[chromo %like% "PYAM"], by="chromo")[, chromo] #add the PYAM sites together
names <- maf[chromo %!in% pyam, .N, by="chromo"][, "chromo"] #get a list of chromosome labels
names_with_pyam <- rbind(names, matrix(nrow=1, ncol=1, "PYAM"), use.names=F)
#fwrite(names_with_pyam, "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld/chromosomes.txt", col.names =F)

__plot the pcangsd results, color by phenotypes in the association__

In [23]:
pca <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/output.pcangsd.cov")

In [ ]:
dim(pca)

In [24]:
#get eigenvalues
ev <-eigen(pca)
ev_dt <- data.table(ev$vectors[,1:10])
setnames(ev_dt, c("pc1", "pc2", "pc3", "pc4", "pc5", "pc6", "pc7", "pc8", "pc9", "pc10"))

In [25]:
#variance explained?
ev$values/sum(ev$values)

[1] 0.210885556 0.006227285 0.006188027 0.006178431 0.006170281 0.006156210
  [7] 0.006136525 0.006120749 0.006100143 0.006092071 0.006088621 0.006085733
 [13] 0.006067177 0.006057354 0.006051042 0.006046759 0.006038278 0.006036200
 [19] 0.006034436 0.006009574 0.006009074 0.006007800 0.006006994 0.006002412
 [25] 0.006001062 0.006000503 0.005995837 0.005993658 0.005988982 0.005979582
 [31] 0.005962752 0.005934171 0.005932039 0.005931601 0.005923389 0.005920369
 [37] 0.005913529 0.005902046 0.005898321 0.005888043 0.005866332 0.005854031
 [43] 0.005841799 0.005834674 0.005832308 0.005827376 0.005826391 0.005801093
 [49] 0.005795495 0.005788146 0.005763565 0.005744137 0.005724283 0.005721111
 [55] 0.005704169 0.005668773 0.005603402 0.005575923 0.005520626 0.005490496
 [61] 0.005292954 0.005273817 0.005250345 0.005215067 0.005207732 0.005183352
 [67] 0.005156018 0.005146994 0.005142300 0.005134883 0.005125624 0.005117904
 [73] 0.005111409 0.005105178 0.005090742 0.005085281 0.005079494 0.005052880
 [79] 0.005043324 0.005032129 0.005018289 0.005002540 0.004980669 0.004961479
 [85] 0.004938959 0.004929439 0.004901744 0.004888660 0.004886747 0.004882833
 [91] 0.004878677 0.004875816 0.004870976 0.004838882 0.004830715 0.004824812
 [97] 0.004818846 0.004810495 0.004801210 0.004795966 0.004789556 0.004758996
[103] 0.004750876 0.004725813 0.004705078 0.004697940 0.004696173 0.004690623
[109] 0.004680488 0.004670783 0.004648239 0.004636661 0.004622637 0.004613630
[115] 0.004605753 0.004600986 0.004590230 0.004569191 0.004561809 0.004558685
[121] 0.004543619 0.004537014 0.004534334 0.004526937 0.004522427 0.004512769
[127] 0.004509655 0.004507771 0.004502826 0.004494850 0.004489577 0.004486402
[133] 0.004483308 0.004480072 0.004471963 0.004471485 0.004463674 0.004461240
[139] 0.004448132 0.004444300 0.004442020 0.004412802 0.004404241 0.004398023
[145] 0.004355561 0.004335499 0.004316104 0.004255849 0.004245531 0.004213902
[151] 0.004176323 0.004091696 0.003954023

__join together the metadata with other covariates and phenotypes with the pcs- be sure to check that the samples are ordered the same as bamlist__

In [26]:
bamlist <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/bamlist.list.txt")
meta <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/Sample_Meta.txt")[is.na(Group), Group :=-999][Group=="High", Group:=1][Group=="Low", Group:=0][, .(sample=gen_id, ucrit=Ucrit, sl=SL, dam, sire, case=Group, capture=Capture)]
sorted_meta <- left_join(bamlist, meta, by="sample")
pca_phenos <- cbind(sorted_meta , ev_dt)[, sequencing_lib := ifelse(capture %in% c("Pop01","Pop02","Pop03", "Pop04", "Pop05", "Pop06", "Pop07","Pop08"), 1, 2)]
#write out the pcs to use as covariates
#fwrite(pca_phenos[, "capture"], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/capture.cov", col.names = F)

Warning message in `[.data.table`(fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/Sample_Meta.txt")[is.na(Group), :
“Coerced double RHS to character to match the type of the target column (column 10 named 'Group'). If the target column's type character is correct, it's best for efficiency to avoid the coercion and create the RHS as type character. To achieve that consider R's type postfix: typeof(0L) vs typeof(0), and typeof(NA) vs typeof(NA_integer_) vs typeof(NA_real_). You can wrap the RHS with as.character() to avoid this warning, but that will still perform the coercion. If the target column's type is not correct, it's best to revisit where the DT was created and fix the column type there; e.g., by using colClasses= in fread(). Otherwise, you can change the column type now by plonking a new column (of the desired type) over the top of it; e.g. DT[, `Group`:=as.double(`Group`)]. If the RHS of := has nrow(DT) elements then the assignment is called a c

In [47]:
pca_unlinked <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/pc1-3_unlinked.cov")

In [50]:
#pca_unlinked$sample <- bamlist$sample
pca_pheno_unlinked <- cbind(pca_unlinked, pca_phenos[,1:3])

In [51]:
head(pca_pheno_unlinked)

V1,V2,V3,sample,ucrit,sl
-0.092607712,0.0174850921,-0.0244912829,Pop01_APPC1611,8.13,4.40
-0.114829109,-0.0122763883,-0.0045606441,Pop01_APPC1917,1.20,4.35
0.003392844,-0.0002162177,0.0002983512,Pop01_APPC1921,7.33,4.28
-0.034442932,-0.9970604955,0.0148471171,Pop01_APPC2029,8.82,4.25
0.022761767,-0.0009952023,0.0006811659,Pop01_APPC2030,7.32,4.54
0.079684748,0.0003954962,0.0004661896,Pop01_APPC2101,5.07,3.94


In [ ]:
filter <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/PCAngsd_Filter2.txt")

In [ ]:
pca_phenos$pcangsd_filter <- filter[,1]

In [ ]:
pca_phenos[, pcangsd_filter :=ifelse(sample %in% pca_phenos[pc2 < -0.75, sample],0, pcangsd_filter)]

In [48]:
head(pca_phenos)

sample,ucrit,sl,dam,sire,case,capture,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,sequencing_lib
Pop01_APPC1611,8.13,4.40,D125,S144,1,Pop01,-0.092670635,-0.0217683252,-0.0823144410,-0.1177519963,0.4325507351,0.815169472,0.304143123,6.085239e-02,-0.0479891950,-0.0103735832,1
Pop01_APPC1917,1.20,4.35,D85,S173,0,Pop01,-0.114995555,-0.0049404088,-0.0084712579,-0.0073748237,0.0151975141,-0.018869047,-0.036858803,-1.936825e-02,0.0220150535,-0.0045099847,1
Pop01_APPC1921,7.33,4.28,D85,S173,1,Pop01,0.003367341,0.0003246282,0.0004035543,0.0004142743,-0.0008168261,0.001172834,0.002222074,-2.361056e-05,-0.0006646217,0.0004675780,1
Pop01_APPC2029,8.82,4.25,D70,S98,1,Pop01,0.022773539,0.0007730465,0.0013156391,0.0010744393,-0.0022055729,0.002976713,0.005192988,2.428480e-03,-0.0018301433,0.0007796293,1
Pop01_APPC2030,7.32,4.54,D70,S98,1,Pop01,0.079804091,0.0005373194,0.0009161695,0.0008095938,-0.0014722499,0.001812070,0.003008333,3.132228e-03,-0.0029753554,0.0003094787,1
Pop01_APPC2101,5.07,3.94,D269,S94,1,Pop01,-0.126313695,-0.0035292362,-0.0047799108,-0.0066165296,0.0102512081,-0.012732940,-0.026813842,-8.009059e-03,0.0123313716,-0.0502423383,1


In [ ]:
nrow(pca_phenos[pcangsd_filter==0])
#pca_phenos$capture <- gsub(pca_phenos$capture, "Pop", "", fixed = T)

In [ ]:
fwrite(pca_phenos[,.(pcangsd_filter)],"/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/PCAngsd_Filter3.txt", col.names = FALSE )

In [ ]:
fwrite(pca_phenos[,.(pc1, pc2, pc3)],"/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/pc1-3_unlinked.cov", col.names = FALSE )

In [54]:
#what pcs/covariates explain the phenotype?
predictors <- as.data.table(summary(glm(data=pca_pheno_unlinked, sl ~ V1+V2+V3))$coeff)
 
predictors$names <- row.names(summary(glm(data=pca_pheno_unlinked, sl ~ V1+V2+V3))$coeff)
setnames(predictors, 4, "pvalue")
predictors[pvalue <0.06][order(pvalue)]

Estimate,Std. Error,t value,pvalue,names
4.2565577,0.01110523,383.293050,2.851757e-217,(Intercept)
-0.2634892,0.13867298,-1.900076,5.943614e-02,V1


In [56]:
predictors <- as.data.table(summary(glm(data=pca_pheno_unlinked, ucrit ~ V1+V2+V3))$coeff)
 
predictors$names <- row.names(summary(glm(data=pca_pheno_unlinked, ucrit ~ V1+V2+V3))$coeff)
setnames(predictors, 4, "pvalue")
predictors[pvalue <0.08][order(pvalue)]

Estimate,Std. Error,t value,pvalue,names
4.077585,0.2397162,17.01005,2.820919e-37,(Intercept)


In [ ]:
summary(aov(data=pca_phenos, pc2~dam))
summary(aov(data=pca_phenos, pc2~sire))
summary(aov(data=pca_phenos, pc2~sl))

In [ ]:
ggplot()+
    geom_boxplot(data=pca_phenos, aes(x=dam, y=pc1))+
    theme(axis.text.x=element_text(angle=90,hjust=1))

In [ ]:
ggplot()+
    geom_boxplot(data=pca_phenos, aes(x=dam, y=ucrit))+
    theme(axis.text.x=element_text(angle=90,hjust=1))

In [ ]:
#ggplot()+
#    geom_point(data=pca_phenos, aes(x=pc1, y=pc2, col=as.factor(case)))

In [ ]:
#ggplot()+
#    geom_point(data=pca_phenos[], aes(x=pc1, y=pc2, col=sl)) +
#    scale_color_viridis(color.palette=magma)

In [ ]:
#ggplot()+
#    geom_point(data=pca_phenos, aes(x=pc1, y=pc2, col=ucrit)) +
#    scale_color_viridis(option="magma")

In [ ]:
#ggplot()+
#    geom_point(data=pca_phenos, aes(x=pc1, y=pc2, col=sequencing_lib)) +
#    scale_color_viridis(option="magma")

In [ ]:
ggplot(data=pca_phenos, aes(x=pc1, y=pc2, col=dam, label=sample))+
    geom_point() #+
    #geom_text(nudge_x = 0.02)

### evaluate model and plot GWAS results for each phenotype

In [174]:
lrt_ucrit <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ucrit_assoc5_maf0.05_noMaxdepth_minQ30_minCount16.lrt0.gz")
lrt_ucrit[, locus := paste(Chromosome, Position, sep=":")]
setnames(lrt_ucrit, "LRTscore", "LRT")

#filter out loci in hwe and NAN
lrt_filt_ucrit <- lrt_ucrit[LRT>0]#[,Chromosome := paste0("chr", Chromosome, sep="")] uncomment this for the top go analysis
#lrt_filt <- lrt[LRT>0]
#format for manhattan plot
#get pvalues
lrt_filt_ucrit$pvalue<-dchisq(lrt_filt_ucrit$LRT, df=1)
lrt_filt_ucrit[, Chromosome := gsub(".","", Chromosome, fixed = T)][, Chromosome := gsub("CM","", Chromosome, fixed = T)][, Chromosome := gsub("PYAM","", Chromosome, fixed = T)]
lrt_filt_ucrit[, Chromosome := as.numeric(Chromosome)]
lrt_filt_ucrit$SNP<-paste("r",1:length(lrt_filt_ucrit$Chromosome), sep="")


lrt_filt_ucrit <- lrt_filt_ucrit[order(LRT)][,quantile:=1:nrow(lrt_filt_ucrit)/nrow(lrt_filt_ucrit)] #quantiles for observed lrt
exp_lrt_ucrit <- data.table( exp_lrt = rchisq(nrow(lrt_filt_ucrit), 1))[order(exp_lrt)]
lrt_filt_ucrit$exp_lrt_ucrit <-exp_lrt_ucrit[, exp_lrt]


In [176]:
tail(lrt_filt_ucrit)

Chromosome,Position,Major,Minor,Frequency,N,LRT,high_WT/HE/HO,LRTem,beta,SE,emIter,locus,pvalue,SNP,quantile,exp_lrt_ucrit
13,1154707,C,A,0.095582,159,13.45818,69/10/0,13.54872,3.192640,1.335931,10,13:1154707,0.00013001982,r71919,0.9999624,16.81349
14,28287841,A,T,0.152335,159,13.69013,29/12/0,93.17074,5.620407,0.158527,37,14:28287841,0.00011479753,r81927,0.9999699,17.73872
6,30026548,A,G,0.182565,159,14.11957,14/12/0,15.34243,-2.815905,0.745015,21,6:30026548,0.00009119550,r34584,0.9999774,19.00107
13,34982957,A,T,0.143409,159,14.54706,30/10/0,15.54311,-3.400393,1.096761,28,13:34982957,0.00007255504,r75803,0.9999849,20.37319
5,37582791,G,T,0.111491,159,15.36749,55/11/0,14.97047,-2.777710,0.987626,3,5:37582791,0.00004683829,r29166,0.9999925,20.73431
24,6021620,T,C,0.139610,159,18.12104,33/13/0,19.33754,-3.541276,1.039912,21,24:6021620,0.00001088639,r129188,1.0000000,21.69824


In [177]:
lrt_sl <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/sl_assoc5_maf0.05_noMaxdepth_unlinkedPC1cov_minQ30_minCount16.lrt0.gz")
lrt_sl[, locus := paste(Chromosome, Position, sep=":")]
#for hybrid assoc 5
#setnames(lrt_sl, "LRTscore", "LRT")
#filter out loci in hwe and NAN
lrt_filt_sl <- lrt_sl[LRT>0]#[,Chromosome := paste0("chr", Chromosome, sep="")]uncomment this for the top go analysis
#lrt_filt <- lrt[LRT>0]
#format for manhattan plot
#get pvalues
lrt_filt_sl$pvalue<-dchisq(lrt_filt_sl$LRT, df=1)
lrt_filt_sl[, Chromosome := gsub(".","", Chromosome, fixed = T)][, Chromosome := gsub("CM","", Chromosome, fixed = T)][, Chromosome := gsub("PYAM","", Chromosome, fixed = T)]
lrt_filt_sl[, Chromosome := as.numeric(Chromosome)]
lrt_filt_sl$SNP<-paste("r",1:length(lrt_filt_sl$Chromosome), sep="")


lrt_filt_sl <- lrt_filt_sl[order(LRT)][,quantile:=1:nrow(lrt_filt_sl)/nrow(lrt_filt_sl)] #quantiles for observed lrt
exp_lrt_sl <- data.table( exp_lrt = rchisq(nrow(lrt_filt_sl), 1))[order(exp_lrt)]
lrt_filt_sl$exp_lrt_sl <-exp_lrt_sl[, exp_lrt]


In [146]:
pdf("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/pval_qq_ucrit.pdf")
qqman::qq(lrt_filt_ucrit$pvalue)
dev.off()

pdf("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/pval_qq_sl.pdf")
qqman::qq(lrt_filt_sl$pvalue)
dev.off()


png 
  2

png 
  2

In [157]:
#compare plots and inflation 
pdf("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/lrt_qq_ucrit.pdf")
qqplot(lrt_filt_ucrit$exp_lrt, lrt_filt_ucrit$LRT, xlab = "Expected LRT", ylab="Observed LRT")
abline(0, 1, col = "red")
dev.off()
#median(lrt_filt_ucrit$LRT) / median(lrt_filt_ucrit$exp_lrt) #GC lambda = ratio at medians
#summary(lrt_filt_ucrit$LRT)

pdf("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/lrt_qq_sl.pdf")
qqplot(lrt_filt_sl$exp_lrt, lrt_filt_sl$LRT, xlab = "Expected LRT", ylab="Observed LRT")
abline(0, 1, col = "red")
dev.off()
#median(lrt_filt_sl$LRT) / median(lrt_filt_sl$exp_lrt) #GC lambda = ratio at medians
#summary(lrt_filt_sl$LRT)


png 
  2

png 
  2

In [93]:
#compare how each phenotype separates from null
lrt_ucrit_null <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/permute/assoc2/cat_permuted_assoc2_ucrit.lrt")
#lrt_ucrit[, locus := paste(Chromosome, Position, sep=":")]

lrt_sl_null <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/permute/assoc2/cat_permuted_assoc2_sl.lrt")
#lrt_sl[, locus := paste(Chromosome, Position, sep=":")]
#filter out loci in hwe and NAN




In [372]:
max(lrt_ucrit_null$LRT)
max(lrt_sl_null$LRT)

[1] 23.34577

[1] 26.2002

In [281]:
max(lrt_filt_ucrit$LRT)
max(lrt_filt_sl$LRT)

[1] 18.12104

[1] 21.31278

__try using the permutations as the expected for the qq plot__

In [159]:
lrt_sl_null <- lrt_sl_null[LRT > 0][order(LRT)][,quantile:=1:nrow(lrt_sl_null[LRT > 0])/nrow(lrt_sl_null[LRT > 0])] #quantiles for observed lrt
pdf("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/lrt_null_qq_sl.pdf")
qqplot(lrt_sl_null$LRT, lrt_filt_sl$LRT, xlab = "Expected LRT", ylab="Observed LRT")
abline(0, 1, col = "red")
dev.off()

png 
  2

In [158]:
lrt_ucrit_null <- lrt_ucrit_null[LRT > 0][order(LRT)][,quantile:=1:nrow(lrt_ucrit_null[LRT > 0])/nrow(lrt_ucrit_null[LRT > 0])] #quantiles for observed lrt
pdf("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/lrt_null_qq_ucrit.pdf")
qqplot(lrt_ucrit_null$LRT, lrt_filt_ucrit$LRT, xlab = "Expected LRT", ylab="Observed LRT")
abline(0, 1, col = "red")
dev.off()

png 
  2

__Get the top 5% of LRT from the null permutations, find snps above that threshold in true association__

In [98]:
0.01 *nrow(lrt_ucrit_null[LRT >0])

[1] 132848

In [99]:
ucrit_null_top <- lrt_ucrit_null[LRT >0][order(-LRT)][1:132848,]

In [100]:
summary(ucrit_null_top$LRT)
candidates_ucrit <- lrt_filt_ucrit[LRT > min(ucrit_null_top$LRT)]
nrow(candidates_ucrit)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  6.305   6.782   7.455   7.970   8.612  24.906 

[1] 503

__Also for SL phenotype__

In [101]:
0.01 *nrow(lrt_sl_null[LRT >0])

[1] 113137.7

In [102]:
sl_null_top <- lrt_sl_null[LRT >0][order(-LRT)][1:113137,]
summary(sl_null_top$LRT)
candidates_sl <- lrt_filt_sl[LRT > min(sl_null_top$LRT)]
nrow(candidates_sl)
summary(candidates_sl$pvalue)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  6.445   6.928   7.614   8.139   8.785  26.200 

[1] 2676

       Min.     1st Qu.      Median        Mean     3rd Qu.        Max. 
0.000002035 0.001395829 0.002951004 0.003004133 0.004573168 0.006259550 

In [305]:

candidates_sl[locus %in% candidates_ucrit$locus]

Chromosome,Position,Major,Minor,Frequency,N,LRT,high_WT/HE/HO,locus,pvalue,SNP,quantile,exp_lrt_sl,rank,padjBH


In [189]:
min(lrt_filt_ucrit$pvalue)
min(lrt_filt_sl$pvalue)

[1] 0.00001088639

[1] 0.000002035062

__Calculate the BH critical threshold__

In [188]:
nrow(lrt_filt_ucrit)
nrow(lrt_filt_sl)

[1] 132834

[1] 105948

In [191]:
head(lrt_filt_ucrit)

Chromosome,Position,Major,Minor,Frequency,N,LRT,high_WT/HE/HO,LRTem,beta,SE,emIter,locus,pvalue,SNP,quantile,exp_lrt_ucrit,rank,padjBH
24,6021620,T,C,0.139610,159,18.12104,33/13/0,19.33754,-3.541276,1.039912,21,24:6021620,0.00001088639,r129188,1.0000000,21.69824,1,0.000004516916
5,37582791,G,T,0.111491,159,15.36749,55/11/0,14.97047,-2.777710,0.987626,3,5:37582791,0.00004683829,r29166,0.9999925,20.73431,2,0.000009033832
13,34982957,A,T,0.143409,159,14.54706,30/10/0,15.54311,-3.400393,1.096761,28,13:34982957,0.00007255504,r75803,0.9999849,20.37319,3,0.000013550748
6,30026548,A,G,0.182565,159,14.11957,14/12/0,15.34243,-2.815905,0.745015,21,6:30026548,0.00009119550,r34584,0.9999774,19.00107,4,0.000018067663
14,28287841,A,T,0.152335,159,13.69013,29/12/0,93.17074,5.620407,0.158527,37,14:28287841,0.00011479753,r81927,0.9999699,17.73872,5,0.000022584579
13,1154707,C,A,0.095582,159,13.45818,69/10/0,13.54872,3.192640,1.335931,10,13:1154707,0.00013001982,r71919,0.9999624,16.81349,6,0.000027101495


In [193]:
lrt_filt_ucrit <- lrt_filt_ucrit[order(pvalue)]
lrt_filt_ucrit$rank=1:NROW(lrt_filt_ucrit) #rank snp by order
lrt_filt_ucrit$padjBH=(lrt_filt_ucrit$rank/nrow(lrt_filt_ucrit))*.2#then calculate the critical by dividing each rank by the total number of snps, and then multiply by fdr
lrt_filt_ucrit_sig=lrt_filt_ucrit[lrt_filt_ucrit$pvalue<lrt_filt_ucrit$padjBH,]
lrt_filt_ucrit_sig

Chromosome,Position,Major,Minor,Frequency,N,LRT,high_WT/HE/HO,LRTem,beta,SE,emIter,locus,pvalue,SNP,quantile,exp_lrt_ucrit,rank,padjBH


In [78]:
lrt_filt_sl <- lrt_filt_sl[order(pvalue)]
lrt_filt_sl$rank=1:NROW(lrt_filt_sl) #rank snp by order
lrt_filt_sl$padjBH=(lrt_filt_sl$rank/nrow(lrt_filt_sl))*.2 #then calculate the critical by dividing each rank by the total number of snps, and then multiply by fdr
lrt_filt_sl_sig=lrt_filt_sl[lrt_filt_sl$pvalue<lrt_filt_sl$padjBH,]
lrt_filt_sl_sig

Chromosome,Position,Major,Minor,Frequency,N,LRT,high_WT/HE/HO,locus,pvalue,rank,padjBH
chr24,16962690,T,G,0.258409,147,20.29089,10/13/0,24:16962690,0.000003476556,2,0.000003775437
chr10,406013,C,T,0.253512,147,14.06247,11/13/0,10:406013,0.000094026875,50,0.000094385925


In [79]:
candidate_sl_bh <- lrt_filt_sl[pvalue < max(lrt_filt_sl_sig$pvalue)]
nrow(candidate_sl_bh[order(Chromosome, Position)] )

[1] 49

In [162]:
lrt_filt_ucrit[, Chromosome := gsub(".","", Chromosome, fixed = T)][, Chromosome := gsub("CM","", Chromosome, fixed = T)][, Chromosome := gsub("PYAM","", Chromosome, fixed = T)]
lrt_filt_ucrit[, Chromosome := as.numeric(Chromosome)]

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”

In [161]:
lrt_filt_sl[, Chromosome := gsub(".","", Chromosome, fixed = T)][, Chromosome := gsub("CM","", Chromosome, fixed = T)][, Chromosome := gsub("PYAM","", Chromosome, fixed = T)]
lrt_filt_sl[, Chromosome := as.numeric(Chromosome)]

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”

In [180]:
gw_sl <- dchisq(min(sl_null_top$LRT), df=1)
gw_sl_null <- dchisq(min(ucrit_null_top$LRT), df=1)
png("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/sl_manhattan.png")
qqman::manhattan(lrt_filt_sl, chr="Chromosome", bp="Position", p="pvalue", genomewideline = -log10(max(lrt_filt_sl_sig$pvalue)), suggestiveline = F)
dev.off()

png 
  2

In [168]:
head(lrt_sl)

Chromosome,Position,Major,Minor,Frequency,N,LRT,high_WT/HE/HO,locus
1,327,G,T,0.147200,147,-999,4/1/0,1:327
1,479,T,A,0.098049,147,-999,15/0/0,1:479
1,480,T,G,0.280927,147,-999,0/2/0,1:480
1,529,C,T,0.070010,147,-999,44/4/0,1:529
1,787,T,C,0.453030,147,-999,0/2/0,1:787
1,1492,T,G,0.077941,147,-999,28/3/0,1:1492


In [ ]:
gw_ucrit_null <- dchisq(min(ucrit_null_top$LRT), df=1)
png("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/ucrit_manhattan.png")
qqman::manhattan(lrt_filt_ucrit, chr="Chromosome", bp="Position", p="pvalue", genomewideline = -log10(gw_ucrit_null), suggestiveline = F)
dev.off()

__write out a list of candidate loci for blast__

In [309]:
candidates_sl_blast <- candidates_sl[,.(locus, Chromosome, start=Position-3000, end=Position+3000)]#, range=paste0(start, end, sep="-"))]

In [311]:
candidates_ucrit_blast <- candidates_ucrit[,.(locus, Chromosome, start=Position-3000, end=Position+3000)]#, range=paste0(start, end, sep="-"))]

In [312]:
candidates_sl_blast[, range:=paste(start, end, sep="-")]

In [313]:
candidates_ucrit_blast[, range:=paste(start, end, sep="-")]

In [314]:
fwrite(candidates_sl_blast[,.(Chromosome, start, end)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/candidates_sl_loci3kbp.bed", sep="\t", col.names = F)
fwrite(candidates_sl_bh[,.(Chromosome, start, end)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/candidates_sl_loci3kbp.bed", sep="\t", col.names = F)

In [315]:
fwrite(candidates_ucrit_blast[,.(Chromosome, start, end)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/candidates_ucrit_loci3kbp.bed", sep="\t", col.names = F)

__write out the candidate positions to find closest feature in percula genome__

In [362]:
candidates_sl_bedops <- candidates_sl[,.(locus, Chromosome, start=Position-100, end=Position+100)]#, range=paste0(start, end, sep="-"))]
candidates_ucrit_bedops <- candidates_ucrit[,.(locus, Chromosome, start=Position-100, end=Position+100)]#, range=paste0(start, end, sep="-"))]
fwrite(candidates_sl_bedops[,.(Chromosome, start, end)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/candidates_sl_loci100bp.bed", sep="\t", col.names = F)
fwrite(candidates_ucrit_bedops[,.(Chromosome, start, end)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/candidates_ucrit_loci100bp.bed", sep="\t", col.names = F)

In [51]:
all_sl_bedops <- lrt_filt_sl[,.(locus, Chromosome, start=Position-50, end=Position+50)][,Chromosome := paste0("chr", Chromosome, sep="")]#, range=paste0(start, end, sep="-"))]
all_ucrit_bedops <- lrt_filt_ucrit[,.(locus, Chromosome, start=Position-50, end=Position+50)][,Chromosome := paste0("chr", Chromosome, sep="")]#, range=paste0(start, end, sep="-"))]
#fwrite(all_sl_bedops[,.(Chromosome, start, end)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/all_sl_loci100bp.bed", sep="\t", col.names = F)
#fwrite(all_ucrit_bedops[,.(Chromosome, start, end)], "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/all_ucrit_loci100bp.bed", sep="\t", col.names = F)

In [37]:
head(all_ucrit_bedops[,Chromosome := paste0("chr", Chromosome, sep="")])

locus,Chromosome,start,end
1:11203219,chr1,11203169,11203269
1:27701448,chr1,27701398,27701498
1:41412714,chr1,41412664,41412764
1:42839370,chr1,42839320,42839420
2:38211704,chr2,38211654,38211754
4:3107432,chr4,3107382,3107482


## blast and bedops in candidate_loci.pynb.

__read in blast results from amarel__

In [344]:
blast_ucrit_zebra <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/zebra_ucrit_candidates_results3kb.out")
blast_header <- c("qseqid", "sseqid", "sstart", "send", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "evalue", "bitscore")
setnames(blast_ucrit_zebra, blast_header)
blast_ucrit_zebra[, db :="zebra"]

blast_ucrit_frenatus <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/afrenatus_ucrit_candidates_results3kb.out")
blast_header <- c("qseqid", "sseqid", "sstart", "send", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "evalue", "bitscore")
setnames(blast_ucrit_frenatus, blast_header)
blast_ucrit_frenatus[, db :="frenatus"]

blast_ucrit <- rbind(blast_ucrit_frenatus, blast_ucrit_zebra)

In [340]:
blast_sl_zebra <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/zebra_sl_candidates_results3kb.out")
blast_header <- c("qseqid", "sseqid", "sstart", "send", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "evalue", "bitscore")
setnames(blast_sl_zebra, blast_header)
blast_sl_zebra[, db :="zebra"]

blast_sl_frenatus <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/afrenatus_sl_candidates_results3kb.out")
blast_header <- c("qseqid", "sseqid", "sstart", "send", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "evalue", "bitscore")
setnames(blast_sl_frenatus, blast_header)
blast_sl_frenatus[, db :="frenatus"]

blast_sl <- rbind(blast_sl_frenatus, blast_sl_zebra)

In [349]:
best_blast_sl <- blast_sl[, .SD[which.max(bitscore)], by =c("qseqid", "db")] #sort by best evalue for each locus
best_blast_sl[, `:=`(sstart=ifelse(sstart > send, send, sstart), send=ifelse(sstart > send, sstart, send))]

In [350]:
best_blast_ucrit <- blast_ucrit[, .SD[which.max(bitscore)], by =c("qseqid", "db")] #sort by best evalue for each locus
best_blast_ucrit[, `:=`(sstart=ifelse(sstart > send, send, sstart), send=ifelse(sstart > send, sstart, send))]

__print out the best and then extract those regions from the corresponding gff in candidate_loci.ipynb__

In [355]:
fwrite(best_blast_sl[db=="frenatus",.(sseqid, sstart, send)],"/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/best_blast_sl_3kbp_frenatus.bed", col.names = F, sep="\t")
fwrite(best_blast_sl[db=="zebra",.(sseqid, sstart, send)],"/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/best_blast_sl_3kbp_zebra.bed", col.names = F, sep="\t")

In [356]:
fwrite(best_blast_ucrit[db=="frenatus",.(sseqid, sstart, send)],"/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/best_blast_ucrit_3kbp_frenatus.bed", col.names = F, sep="\t")
fwrite(best_blast_ucrit[db=="zebra",.(sseqid, sstart, send)],"/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/best_blast_ucrit_3kbp_zebra.bed", col.names = F, sep="\t")

__read the extracted gff files back in to narrow down on the best and get the go annotations for enrichment__

In [211]:
blast_sl <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/zebra_sl_candidates_results.out")
best_blast_sl <- blast_sl[, .SD[which.min(evalue)], by ="qseqid"] #sort by best evalue for each locus
best_blast_sl[, `:=`(sstart=ifelse(sstart > send, send, sstart), send=ifelse(sstart > send, sstart, send))]
head(best_blast_sl)
nrow(best_blast_sl)

qseqid,sseqid,sstart,send,pident,length,mismatch,gapopen,qstart,qend,evalue,bitscore
2:5427895-5437894,CHR_ALT_CTG8_1_11,26631634,26631715,86.747,83,10,1,7448,7530,1.39e-13,91.6
2:23958024-23968023,16,20607110,20607141,100.000,32,0,0,835,866,3.92e-04,60.2
3:14330725-14340724,3,56308670,56308830,87.654,162,19,1,5019,5180,1.73e-42,187.0
5:26591316-26601315,16,55120665,55120876,78.605,215,39,4,883,1093,6.34e-27,135.0
6:1316877-1326876,6,55332457,55332555,84.158,101,12,4,7111,7209,1.08e-14,95.3
7:5223068-5233067,16,52608493,52608671,82.123,179,32,0,8236,8414,1.75e-32,154.0


[1] 24

In [389]:
head(geneList)
class(geneList)

1095_s_at   1130_at   1196_at 1329_s_at 1340_s_at 1342_g_at 
1.0000000 1.0000000 0.6223795 0.5412240 1.0000000 1.0000000

[1] "numeric"

In [387]:
head(ALL)

ExpressionSet (storageMode: lockedEnvironment)
assayData: 1 features, 128 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: 01005 01010 ... LAL4 (128 total)
  varLabels: cod diagnosis ... date last seen (21 total)
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
  pubMedIds: 14684422 16243790 
Annotation: hgu95av2 

In [ ]:
#okay so to use topgo, I need to head a table with row names as genes and col 1 pvalue (optional) and then a list of go annotations where the list name is a gene id and the GO follow as character vector. Get the latter first, sounds harder

In [39]:
#load in the gff file from closest-feature
gff_names <- c("seqid", "start","locus", "feature_start", "feature_end", "feature_id", "score", "strand", "unkn", "feature_type", "phase", "attributes")
sl_nearest <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/sorted_all_sl_100bp_percula_nearest.bed", sep="\t", fill = T)
setnames(sl_nearest, gff_names)

In [40]:
head(sl_nearest)

seqid,start,locus,feature_start,feature_end,feature_id,score,strand,unkn,feature_type,phase,attributes
chr1,9954,10054|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|16508"
chr1,32266,32366|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0"
chr1,32276,32376|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0"
chr1,32300,32400|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0"
chr1,32847,32947|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0"
chr1,33169,33269|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0"


In [41]:
#split the attributs column to get the gene name and the go terms
split <- sl_nearest[, c('gene_id', 'protein_family', 'go_term_dist'):= tstrsplit(attributes, ";")][, c('go_term', 'dist_bp'):= tstrsplit(go_term_dist, "|", fixed=T),by="gene_id"]

In [42]:
head(split)

seqid,start,locus,feature_start,feature_end,feature_id,score,strand,unkn,feature_type,phase,attributes,gene_id,protein_family,go_term_dist,go_term,dist_bp
chr1,9954,10054|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|16508",ID=APERC1_00013921,"Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546",Ontology_term=GO:0005515|16508,Ontology_term=GO:0005515,16508
chr1,32266,32366|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0",ID=APERC1_00013921,"Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546",Ontology_term=GO:0005515|0,Ontology_term=GO:0005515,0
chr1,32276,32376|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0",ID=APERC1_00013921,"Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546",Ontology_term=GO:0005515|0,Ontology_term=GO:0005515,0
chr1,32300,32400|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0",ID=APERC1_00013921,"Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546",Ontology_term=GO:0005515|0,Ontology_term=GO:0005515,0
chr1,32847,32947|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0",ID=APERC1_00013921,"Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546",Ontology_term=GO:0005515|0,Ontology_term=GO:0005515,0
chr1,33169,33269|chr1,26561,55859,APERC1_00013921,.,-,maker,gene,.,"ID=APERC1_00013921;Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546;Ontology_term=GO:0005515|0",ID=APERC1_00013921,"Dbxref=InterPro:IPR001073,InterPro:IPR011489,Pfam:PF00386,Pfam:PF07546",Ontology_term=GO:0005515|0,Ontology_term=GO:0005515,0


In [44]:
go_sl <- split[!is.na(gene_id), .(gene_id=gsub("ID=", "", gene_id), go_term=gsub("Ontology_term=", "", go_term))][, go_term:=gsub(",", " ", go_term)][!is.na(go_term)]
go_sl <- unique(go_sl, by="gene_id")
go_terms <- by(go_sl, go_sl$gene_id,function(x) setNames(x$go_term,x$name))
head(go_sl)

gene_id,go_term
APERC1_00013921,GO:0005515
APERC1_00013924,GO:0005576 GO:0009755
APERC1_00013925,GO:0004252 GO:0006508
APERC1_00013926,GO:0004252 GO:0006508
APERC1_00013937,GO:0005509
APERC1_00013938,GO:0016616 GO:0055114


In [49]:
go_sl <- split[!is.na(gene_id), .(gene_id=gsub("ID=", "", gene_id), go_term=gsub("Ontology_term=", "", go_term))][, go_term:=gsub(",", " ", go_term)][!is.na(go_term)]
go_sl_vec <- go_sl[, setNames(go_term, gene_id)]


In [541]:
sl_locus_lrt <- left_join(candidates_sl_bedops, lrt_filt_sl)[, .(Chromosome, start, end, LRT)]

Joining, by = c("locus", "Chromosome")

In [64]:
sl_locus_lrt <- left_join(all_sl_bedops, lrt_filt_sl[pvalue>1, pvalue:=1])[, .(Chromosome, start, end, pvalue)]
genes_score <- left_join(sl_locus_lrt, split[, .(seqid, start, feature_id)], by="start")[, .(feature_id, pvalue)][!is.na(feature_id)]
genes_score <- unique(genes_score, by="feature_id")
genes_score_vec <- genes_score[, setNames(pvalue, feature_id)]

Joining, by = c("locus", "Chromosome")

In [135]:
#make the topGo object
sig <- max(candidates_sl$pvalue)
#sig <- max(lrt_filt_sl_sig$pvalue)
GOdata <- new("topGOdata", description="go analysis of closest genes for a percula phenotype of sl", ontology="BP", allGenes=genes_score_vec, geneSel = function(foo) {foo < sig}, annotationFun=annFUN.gene2GO, gene2GO=go_sl_vec)


Building most specific GOs .....
	( 83 GO terms found. )

Build GO DAG topology ..........
	( 437 GO terms and 830 relations. )

Annotating nodes ...............
	( 308 genes annotated to the GO terms. )


In [136]:
test.stat <- new("classicScore", testStatistic = GOKSTest, name = "KS tests")
resultKS <- getSigGroups(GOdata, test.stat)

test.stat <- new("classicCount", testStatistic = GOFisherTest, name = "Fisher test")
resultFisher <- getSigGroups(GOdata, test.stat)

test.stat <- new("weightCount", testStatistic = GOFisherTest, name = "Fisher test", sigRatio = "ratio")
resultWeight <- getSigGroups(GOdata, test.stat)


			 -- Classic Algorithm -- 

		 the algorithm is scoring 437 nontrivial nodes
		 parameters: 
			 test statistic: KS tests
			 score order: increasing

			 -- Classic Algorithm -- 

		 the algorithm is scoring 0 nontrivial nodes
		 parameters: 
			 test statistic: Fisher test
Warning message in .local(object, test.stat, ...):
“No enrichment can pe performed - there are no feasible GO terms!”
			 -- Weight Algorithm -- 

		 The algorithm is scoring 0 nontrivial nodes
		 parameters: 
			 test statistic: Fisher test : ratio
Warning message in getSigGroups(GOdata, test.stat):
“No enrichment can pe performed - there are no feasible GO terms!”

In [139]:
allRes <- GenTable(GOdata, classic = resultFisher, KS = resultKS, weight = resultWeight, orderBy = "KS", ranksOf = "classic", topNodes = 40)

In [140]:
head(allRes)

GO.ID,Term,Annotated,Significant,Expected,Rank in classic,classic,KS,weight
GO:0034622,cellular protein-containing complex asse...,3,0,0,1,1,0.052,1
GO:0006399,tRNA metabolic process,2,0,0,2,1,0.068,1
GO:0008033,tRNA processing,2,0,0,3,1,0.068,1
GO:0051276,chromosome organization,3,0,0,4,1,0.131,1
GO:0006996,organelle organization,10,0,0,5,1,0.133,1
GO:0070071,proton-transporting two-sector ATPase co...,2,0,0,6,1,0.141,1


In [126]:
fwrite(allRes, "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/GO_table_BH_cutoff_sl.csv")

__for ucrit__

In [132]:
#load in the gff file from closest-feature
gff_names <- c("seqid", "start","locus", "feature_start", "feature_end", "feature_id", "score", "strand", "unkn", "feature_type", "phase", "attributes")
ucrit_nearest <- fread("/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/sorted_all_ucrit_100bp_percula_nearest.bed", sep="\t", fill = T)
setnames(ucrit_nearest, gff_names)

split <- ucrit_nearest[, c('gene_id', 'protein_family', 'go_term_dist'):= tstrsplit(attributes, ";")][, c('go_term', 'dist_bp'):= tstrsplit(go_term_dist, "|", fixed=T),by="gene_id"]

ucrit_locus_lrt <- left_join(all_ucrit_bedops, lrt_filt_ucrit[pvalue>1, pvalue:=1])[, .(Chromosome, start, end, pvalue)]
genes_score <- left_join(ucrit_locus_lrt, split[, .(seqid, start, feature_id)], by="start")[, .(feature_id, pvalue)][!is.na(feature_id)]
genes_score <- unique(genes_score, by="feature_id")
genes_score_vec <- genes_score[, setNames(pvalue, feature_id)]

go_ucrit <- split[!is.na(gene_id), .(gene_id=gsub("ID=", "", gene_id), go_term=gsub("Ontology_term=", "", go_term))][, go_term:=gsub(",", " ", go_term)][!is.na(go_term)]
go_ucrit_vec <- go_ucrit[, setNames(go_term, gene_id)]

sig <- max(candidates_ucrit$pvalue)
#sig <- max(lrt_filt_ucrit_sig$pvalue)
GOdata <- new("topGOdata", description="go analysis of closest genes for a percula phenotype of ucrit", ontology="BP", allGenes=genes_score_vec, geneSel = function(foo) {foo < sig}, annotationFun=annFUN.gene2GO, gene2GO=go_ucrit_vec)

test.stat <- new("classicScore", testStatistic = GOKSTest, name = "KS tests")
resultKS <- getSigGroups(GOdata, test.stat)

test.stat <- new("classicCount", testStatistic = GOFisherTest, name = "Fisher test")
resultFisher <- getSigGroups(GOdata, test.stat)

test.stat <- new("weightCount", testStatistic = GOFisherTest, name = "Fisher test", sigRatio = "ratio")
resultWeight <- getSigGroups(GOdata, test.stat)

allRes <- GenTable(GOdata, classic = resultFisher, KS = resultKS, weight = resultWeight, orderBy = "classic", ranksOf = "classic", topNodes = 40)

Joining, by = c("locus", "Chromosome")
Building most specific GOs .....
	( 88 GO terms found. )

Build GO DAG topology ..........
	( 454 GO terms and 869 relations. )

Annotating nodes ...............
	( 335 genes annotated to the GO terms. )

			 -- Classic Algorithm -- 

		 the algorithm is scoring 454 nontrivial nodes
		 parameters: 
			 test statistic: KS tests
			 score order: increasing

			 -- Classic Algorithm -- 

		 the algorithm is scoring 11 nontrivial nodes
		 parameters: 
			 test statistic: Fisher test

			 -- Weight Algorithm -- 

		 The algorithm is scoring 11 nontrivial nodes
		 parameters: 
			 test statistic: Fisher test : ratio

	 Level 6:	1 nodes to be scored.

	 Level 5:	1 nodes to be scored.

	 Level 4:	1 nodes to be scored.

	 Level 3:	3 nodes to be scored.

	 Level 2:	4 nodes to be scored.

	 Level 1:	1 nodes to be scored.


In [134]:
fwrite(allRes, "/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/results/GO_table_null_cutoff_ucrit.csv")

In [ ]:
Proportion of variance in phenotype explained by a given SNP (PVE) = [2*(beta^2)*MAF*(1-MAF)]/[2*(beta^2)*MAF(1-MAF)+((se(beta))^2)*2*N*MAF*(1-MAF)]
where:
N - sample size
se(beta) - standard error of effect size for the genetic variant of interest
beta - effect size for the genetic variant of interest
MAF - minor allele frequency for the genetic variant of interest
it is described in:
Shim, H., Chasman, D.I., Smith, J.D., Mora, S., Ridker, P.M., Nickerson, D.A., Krauss, R.M., and Stephens, M. (2015). A multivariate genome-wide association analysis of 10 LDL subfractions, and their response to statin treatment, in 1868 Caucasians. PLoS One 10, e0120758.
Supplementary Information: S1: Computing proportion of variance in phenotype explained by a given SNP (PVE).